In [1]:
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
from numpy.fft import fft
from scipy.linalg import eigh
import random
from networkx.generators.community import stochastic_block_model
from networkx.linalg.graphmatrix import adjacency_matrix
from numpy.linalg import norm
from matplotlib import cm
import time
# set random seeds
# random.seed(11)
# np.random.seed(11)

In [2]:
class MFSC:

    def __init__(self, size_of_clusters, probs_of_sbm, max_freq, num_of_clusters):

        self.graph = np.asarray(adjacency_matrix(stochastic_block_model(size_of_clusters, probs_of_sbm)).todense())
        self.true_angles_by_cluster = dict()
        self.size_of_clusters = size_of_clusters
        self.N = size_of_clusters.sum()
        self.max_freq = max_freq
        self.num_of_clusters = num_of_clusters
        self.alpha_struc = None
        self.A_k = np.zeros((self.max_freq, self.N, self.N), dtype='complex_')
        self.Phi = np.zeros((self.max_freq, self.num_of_clusters, self.N), dtype='complex_')
        self.d_alpha = 2 * np.pi / self.max_freq
        self.Pi = np.eye(self.N)
        self.Rf = np.zeros((self.max_freq, self.num_of_clusters, self.N), dtype='complex_')
        self.cluster_id = None
        self.estimated_angles = None

    def generate_angles(self):

        self.alpha_struc = np.random.randint(self.max_freq, size=(self.N, self.N))
        self.alpha_struc[np.tril_indices(self.N)] = 0
        self.alpha_struc = self.alpha_struc + np.mod(-self.alpha_struc.T, self.max_freq)
        start_idx, end_idx = 0, 0
        for idx, b_size in enumerate(self.size_of_clusters):
            end_idx += b_size
            alphas = np.random.randint(self.max_freq, size=b_size)
            self.true_angles_by_cluster.setdefault(idx, alphas)
            self.alpha_struc[start_idx:end_idx, start_idx:end_idx] = np.mod(
                alphas.reshape(-1, 1) - alphas,
                self.max_freq
            )
            start_idx += b_size

        return None

    def construct_A_k(self):

        for freq in range(self.max_freq // 2 + 1):
            self.A_k[freq, :, :] = self.graph * np.exp(self.alpha_struc * freq * 1j * self.d_alpha)
            _, eigen_vec = eigh(
                self.A_k[freq, :, :],
                subset_by_index=[self.N - self.num_of_clusters, self.N - 1]
            )
            self.Phi[freq, :, :] = eigen_vec.T
            if freq > 0:
                self.A_k[self.max_freq - freq, :, :] = np.conjugate(self.A_k[freq, :, :])
                self.Phi[self.max_freq - freq, :, :] = np.conjugate(eigen_vec.T)

        return None

    def mfcpqr(self):

        R = self.Phi.copy()
        for m in range(self.num_of_clusters):
            norm_sum = norm(R[:, m:, m:], axis=1).sum(axis=0)
            piv = np.argmax(norm_sum) + m
            R[:, :, [piv, m]] = R[:, :, [m, piv]]
            self.Pi[:, [piv, m]] = self.Pi[:, [m, piv]]
            for freq in range(self.max_freq // 2 + 1):
                _, tRm = MFSC.qr_method_one_step(R[freq, m:, m:])
                R[freq, m:, m:] = tRm
                if freq > 0:
                    R[self.max_freq - freq, m:, m:] = np.conjugate(tRm)

        self.Rf = R @ self.Pi.T

        fft_R = np.abs(fft(self.Rf, axis=0))
        self.cluster_id = np.argmax(np.max(fft_R, axis=0), axis=0)
        self.estimated_angles = np.argmax(fft_R, axis=0)[self.cluster_id, np.array(range(self.N))]

        return None

    @staticmethod
    def qr_method_one_step(X):

        m, _ = X.shape
        r = X[:, 0]
        alpha = -np.exp(1j * np.angle(r[0])) * norm(r)
        e1 = np.zeros(m)
        e1[0] = 1
        u = r - alpha * e1
        v = u / norm(u)
        Q = np.eye(m) - 2 * v.reshape(-1, 1) @ np.conjugate(v).reshape(1, -1)
        R = Q @ X
        R[0, :] = np.exp(-1j * np.angle(R[0, 0])) * R[0, :]

        return Q, R
        

In [3]:
n_sample = 20
b_sizes = np.array([500, 500])
n = b_sizes.sum()
M = 2
max_freq = 33
d_alpha = 2 * np.pi / max_freq
success_rate = np.zeros((41, 16))
angle_error = np.zeros((41, 16))

In [4]:
for i in range(2, 16):
    for j in range(2, 41):
        start = time.perf_counter()
        for t in range(n_sample):
            p = i * np.log(n) / n
            q = j * np.log(n) / n
            sbm_probs = q * np.ones((M, M)) + (p - q) * np.eye(M)
            mfsc = MFSC(
                size_of_clusters=b_sizes,
                probs_of_sbm=sbm_probs,
                max_freq=max_freq,
                num_of_clusters=M
            )
            mfsc.generate_angles()
            mfsc.construct_A_k()
            mfsc.mfcpqr()

            if (mfsc.cluster_id[:b_sizes[0]].sum() == 0 and mfsc.cluster_id[b_sizes[0]:].sum() == b_sizes[1]) \
                    or (
                    mfsc.cluster_id[:b_sizes[0]].sum() == b_sizes[0] and mfsc.cluster_id[b_sizes[0]:].sum() == 0):
                success_rate[j, i] += 1

            angle_error_cluster = np.zeros(M)
            for m in range(M):
                tmp1 = np.exp(-1j*mfsc.estimated_angles[m * 500:(m + 1) * 500]*d_alpha)
                tmp2 = np.exp(1j*mfsc.true_angles_by_cluster[m]*d_alpha)
                d_ang = np.angle((tmp1*tmp2).mean())
                angle_diff = np.abs(np.mod(mfsc.estimated_angles[m * 500:(m + 1) * 500]*d_alpha + d_ang, 2*np.pi)
                                    - mfsc.true_angles_by_cluster[m]*d_alpha)
                angle_error_cluster[m] = np.min(np.vstack((angle_diff, 2*np.pi - angle_diff)), axis=0).max()
                
                
#                 angle_estimate = mfsc.estimated_angles[m * 500:(m + 1) * 500]
#                 angle_true = mfsc.true_angles_by_cluster[m]
#                 tmp = np.zeros(max_freq)
#                 for freq in range(max_freq):
#                     angle_diff = np.abs(np.mod(angle_estimate + freq, max_freq) - angle_true)
#                     tmp[freq] = np.min(np.vstack((angle_diff, max_freq - angle_diff)), axis=0).max()
#                 angle_error_cluster[m] = np.min(tmp)

#             angle_error[j, i] += np.max(angle_error_cluster) * d_alpha
            angle_error[j, i] += np.max(angle_error_cluster)

        print(i, j, angle_error[j, i] / n_sample, success_rate[j, i] / n_sample)
        print("cost %s second" % (time.perf_counter() - start))

5 2 9.71445146547012e-16 1.0
cost 103.074344 second
5 3 9.43689570931383e-16 1.0
cost 104.754435 second
5 4 9.2148511043888e-16 1.0
cost 103.35412360000004 second
5 5 0.09510502592952634 0.95
cost 103.85002650000001 second
5 6 0.17127723395798564 0.85
cost 103.36724959999998 second
5 7 0.019002783912231907 0.95
cost 103.74755589999995 second
5 8 0.15220352510800783 0.9
cost 104.6828246 second


KeyboardInterrupt: 

In [ ]:
plt.figure()
plt.imshow(success_rate[2:41, 2:16][::-1, :] / n_sample, aspect='0.3', extent=[2, 15, 2, 40], cmap='gray')
plt.xlabel(r'$\alpha$')
plt.ylabel(r'$\beta$')
plt.colorbar()
plt.grid(linestyle = '--')
plt.show()
plt.savefig("exact_recovery_MFCPQR")

In [ ]:
initial_cmap = cm.get_cmap('gray')
reversed_cmap=initial_cmap.reversed()
plt.figure()
plt.imshow(angle_error[2:41, 2:16][::-1, :] / n_sample, aspect='0.3', extent=[2, 15, 2, 40], cmap=reversed_cmap)
plt.xlabel(r'$\alpha$')
plt.ylabel(r'$\beta$')
plt.colorbar()
plt.grid(linestyle = '--')
plt.show()
plt.savefig("angle_recovery_MFCPQR")